<a href="https://colab.research.google.com/github/achyutdipu/sciencefaircode2025/blob/main/Python_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

Uploading files

In [11]:
# from google.colab import files
# uploaded = files.upload()
# !unzip dataset-resized.zip

# Data Cleaning

Data Cleaning of Databases, TrashNet database here.

In [12]:
import os
def getpath(path:str):
  while not os.path.exists(path):
    os.makedirs(path)
getpath("science_fair")
def movefiles(currpath:str, tobemoved:str, conv:str, numdigits:int=4, jpg:bool=True):
  getpath("science_fair/class_" + tobemoved + "/")
  for i in range(1, 10**numdigits):
    j = str(i)
    k = j
    while len(k) < 5:
      k = "0" + k
    if jpg:
      if os.path.isfile(currpath + "/" + conv + j + ".jpg"):
        os.rename(currpath + "/" + conv + j + ".jpg", "science_fair/class_" + tobemoved + "/" + tobemoved + "_image_" + k + ".jpg")
      else:
        continue
    else:
      if os.path.isfile(currpath + "/" + conv + j + ".png"):
        os.rename(currpath + "/" + conv + j + ".png", "science_fair/class_" + tobemoved + "/" + tobemoved + "_image_" + k + ".jpg")
      else:
        continue
def movefileslst(currpathlst:list, tobemovedlst:list, convs:list, numdigits:int=4, jpg:bool=True):
  for i in range(0, len(currpathlst)):
    movefiles(currpathlst[i], tobemovedlst[i], convs[i], numdigits=numdigits, jpg=jpg)
dataset1 = ["dataset1/cardboard", "dataset1/glass", "dataset1/metal", "dataset1/paper", "dataset1/plastic", "trash"]
dataset1convs = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
tobemoved1 = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
movefileslst(dataset1, tobemoved1, dataset1convs, numdigits=3)

# AI

AI creation with all functions.

In [10]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import pandas
import tensorflow
from keras.utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing import image_dataset_from_directory
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

def get_imlist(path:str):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png') or f.endswith('.jpeg')]
print("Something")
# print(get_imlist('/home/chronos/u-c38b9bb5f5aad02dfbb7c2d350cb341e3e9feabe/MyFiles/Downloads/media/archive/dataset-resized.zip/dataset-resized/trash'))
def get_dataset(path:str, height:int, same:bool=True, width:int=480) -> tensorflow.data.Dataset: # path to directory must be sorted with folders
    if same:
      im = image_dataset_from_directory(path, color_mode='rgb', image_size=(height, height), labels='inferred')
    else:
      im = image_dataset_from_directory(path, color_mode='rgb', image_size=(width, width), labels='inferred')
    return im
def parse_dataset(dataset:tensorflow.data.Dataset) -> list:
  print(dataset)
  X = []
  Y = []
  for index, row in dataset:
    X.append(list(dataset.resize((128, 128))).flatten() / 255.0)
    Y.append(row[1])
  X = np.array(X)
  Y = to_categorical(Y, 2)
  X = X.reshape(-1, 128, 128, 3)
  X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)
  return [X_train, X_val, Y_train, Y_val]
def CNN() -> tensorflow.keras.Sequential: # building cnn
  model = Sequential()
  model.add(Conv2D(filters = 32, kernel_size = (5,5), padding='valid', activation='relu', data_format='channels_last'))
  model.add(Conv2D(filters = 32, kernel_size = (5,5), padding='valid', activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(2, activation='softmax'))
  return model
model = CNN()
model.summary()
def fine_tuning(model:tensorflow.keras.Sequential=model) -> tensorflow.keras.callbacks.EarlyStopping: # fine tuning model
  optimizer = RMSprop(learning_rate=0.0005, rho=0.9, epsilon=1e-08, weight_decay=0.0)
  model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])
  early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=2, verbose=0, mode='auto')
  return early_stopping
early_stopping = fine_tuning(model)
# lst = parse_dataset(get_dataset("science_fair", 480, same=True))
lst=[1, 2, 3, 4]
def model_training(model:tensorflow.keras.Sequential=model, epochs:int=30, batch_size:int=100, training_list:list=lst, early_stopping:tensorflow.keras.callbacks.EarlyStopping=early_stopping, verbose:int=2) -> tensorflow.keras.callbacks.History:
  history = model.fit(training_list[0], training_list[2], batch_size = batch_size, epochs = epochs, validation_data = (training_list[1], training_list[3]), verbose = 2, callbacks=[early_stopping])
  return history

Something


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)